In [50]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import cactus_dataset


In [51]:
use_gpu = torch.cuda.is_available()
if use_gpu:
    print("Using CUDA")

Using CUDA


In [52]:
data_dir = '../../../data/train/train'
csv_file = '../../../data/train.csv'
TRAIN = 'train'
VAL = 'val'
TEST = 'test'

np.random.seed(0)
image_dataset = cactus_dataset.CactusDataset(csv_file, data_dir)
# split into train, eval, test
train_size = int(0.8 * len(image_dataset))
eval_size = int(0.1 * len(image_dataset))
test_size = len(image_dataset) - train_size - eval_size
train_dataset, eval_dataset, test_dataset = torch.utils.data.random_split(image_dataset, [train_size, eval_size, test_size])

image_datasets = {TRAIN: train_dataset, VAL: eval_dataset, TEST: test_dataset}

dataloaders = {x: DataLoader(image_datasets[x], batch_size=1,
                                             shuffle=True, num_workers=0)
              for x in [TRAIN, VAL, TEST]}

dataset_sizes = {x: len(image_datasets[x]) for x in [TRAIN, VAL, TEST]}
print(dataset_sizes)
class_names = {0: 'No Cactus', 1: 'Cactus'}

inputs, classes = next(iter(dataloaders[TRAIN]))

n_features = inputs.size()[1]

print(n_features)

{'train': 14000, 'val': 1750, 'test': 1750}
3072


In [53]:
# Create model
# f = wx + b, sigmoid at the end
class LogisticRegression(nn.Module):

    def __init__(self, n_input_features):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(n_input_features, 1)

    def forward(self, x):
        y_predicted = torch.sigmoid(self.linear(x))
        return y_predicted
    
model = LogisticRegression(n_features)

In [54]:
# Loss and optimizer
learning_rate = 0.01
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [55]:
X_train = []
y_train = []
X_val = []
y_val = []
for inputs, labels in dataloaders[TRAIN]:
    X_train.append(inputs)
    y_train.append(labels)
for inputs, labels in dataloaders[VAL]:
    X_val.append(inputs)
    y_val.append(labels)
X_train = torch.cat(X_train, 0)
y_train = torch.cat(y_train, 0)
X_val = torch.cat(X_val, 0)
y_val = torch.cat(y_val, 0)

print(X_train.size())
print(y_train.size())

torch.Size([14000, 3072])
torch.Size([14000])


In [ ]:
# training loop
num_epochs = 10000

for epoch in range(num_epochs):
    # forward pass and loss
    y_predicted = model(X_train)
    #make y_predicted and y_train of same shape
    y_predicted = y_predicted.view(-1)
    y_train = y_train.view(-1).float()
    loss = criterion(y_predicted, y_train)
    
    # backward pass
    loss.backward()
    
    # updates
    optimizer.step()
    
    # zero gradients
    optimizer.zero_grad()
    
    if (epoch+1) % 10 == 0:
        print(f'epoch: {epoch+1}, loss = {loss.item():.4f}')
        with torch.no_grad():
            y_predicted = model(X_val)  # no need to call model.forward()
            print(y_predicted)
            y_predicted_cls = y_predicted.round()   # round off to nearest class
            #flatten
            y_val = y_val.view(-1)
            y_predicted_cls = y_predicted_cls.view(-1)
            acc = (y_predicted_cls == y_val).sum() / y_val.shape[0]   # accuracy
            print(acc)

tensor([[0.9247],
        [0.7885],
        [0.9842],
        ...,
        [0.9738],
        [0.8662],
        [0.9464]])
tensor(0.8543)
